In [1]:
import numpy as np 
import pandas as pd 
from subprocess import check_output
#print(check_output(["ls", "../input/"]))
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from os.path import join as opj
from matplotlib import pyplot as plt
import tensorflow as tf
import os
%matplotlib inline


In [2]:
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation,Add, ZeroPadding2D ,AveragePooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers,regularizers
from keras.optimizers import Adam,SGD
from keras.initializers import glorot_uniform
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K
K.set_image_data_format('channels_last')
#K.set_learning_phase(1)

Using TensorFlow backend.


In [3]:
train = pd.read_json("input/train.json")
#test = pd.read_json("input/test.json")

In [12]:
only_band1 = ['band1_'+str(i) for i in range(75*75)]
only_band2 = ['band2_'+str(i) for i in range(75*75)]
both_bands = only_band1+only_band2

In [ ]:
# df = pd.concat([pd.DataFrame(test['band_1'].values.tolist(),columns = only_band1),
#                 pd.DataFrame(test['band_2'].values.tolist(),columns = only_band2)],axis = 1)

In [13]:
def collect_test():
    test_data = pd.DataFrame()
    for i in range(4):
        test_piece = pd.read_csv('input/test_subparts/test'+str(i+1)+'.csv')
        test_data = pd.concat([test_data,test_piece])
        test_data = test_data.reset_index().drop(['index'],axis = 1)
    return test_data

In [14]:
test_data = collect_test()

In [15]:
X_band_test_1=np.array(test_data[only_band1]).reshape(8424,75,75)
X_band_test_2=np.array(test_data[only_band2]).reshape(8424,75,75)
X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis], X_band_test_2[:, :, :, np.newaxis], ((X_band_test_1+X_band_test_2)/2)[:, :, :, np.newaxis]], axis=-1)

In [4]:
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis], X_band_2[:, :, :, np.newaxis],((X_band_1+X_band_2)/2)[:, :, :, np.newaxis]], axis=-1)
target_train=train['is_iceberg']

In [ ]:
# X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
# X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
# X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis], X_band_test_2[:, :, :, np.newaxis], ((X_band_test_1+X_band_test_2)/2)[:, :, :, np.newaxis]], axis=-1)

In [ ]:

X_train_cv, X_valid, y_train_cv, y_valid = train_test_split(X_train, target_train, random_state=2,stratify = target_train, train_size=0.80)

### Cross Validation Folds

In [5]:
def data_augmentation(X_input,Y_input,batch_size = 32):
    data_aug = ImageDataGenerator(#featurewise_center=True,
                             #featurewise_std_normalization=True,
                             #zca_whitening=True,
                             rotation_range=0,
                             width_shift_range = 0,
                             height_shift_range = 0,
                             zoom_range = 0,
                             data_format = 'channels_last',
                             horizontal_flip = True,
                             vertical_flip = True)#,fill_mode = 'constant',cval = 0)
    data_aug_batches = data_aug.flow(X_input,Y_input,batch_size = batch_size)
    return data_aug_batches


In [6]:

def cnnmodel():
    X_input = Input(shape = (75,75,3))
    #First composite layer
    X = Conv2D(128,kernel_size = (5,5))(X_input)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(3,3),strides =(2,2))(X)
    X = Dropout(rate = 0.2)(X)
    
    #Second composite layer
    X = Conv2D(256,kernel_size = (3,3))(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(3,3),strides =(2,2))(X)
    X = Dropout(rate = 0.2)(X)
    
    #Third composite layer
    X = Conv2D(256,kernel_size = (3,3))(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(2,2),strides =(2,2))(X)
    X = Dropout(rate = 0.2)(X)

    #Forth composite layer
    X = Conv2D(128,kernel_size = (3,3))(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(3,3),strides =(2,2))(X)
    X = Dropout(rate = 0.2)(X)

    #Flatten layer
    X = Flatten()(X)
    
    #First dense layer
    X = Dense(512,activation = 'relu')(X)
    X = Dropout(rate = 0.2)(X)
   
    #Second dense layer
    X = Dense(256,activation = 'relu')(X)
    X = Dropout(rate = 0.2)(X)
   
    #Decision layer
    X = Dense(1,activation='sigmoid')(X)
    
    model = Model(inputs=X_input,outputs=X)

    return model

def model_compile(model,lr = 0.0001,decay = 0, freezing_layers = None,weights_path = None):
   
    if weights_path:
        model.load_weights(weights_path)
    else:
        pass
    if freezing_layers:
        for layer in model.layers[:freezing_layers]:
            layer.trainable = False
    else:
        pass
    optimizer = Adam(lr = lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=decay)
    #optimizer = SGD(lr, decay=0, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])
    model.summary()
    return model

def get_callbacks(filepath = ".model_weights.hdf5", patience=7, monitor = 'val_loss', verbose = 1, save_best_only = True):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, monitor=monitor, verbose = verbose, save_best_only=True)
    return [es,msave]


In [7]:

def normal_fit(model,callbacks,epochs = 100,batch_size = 32,verbose = 1):
    
    result = model.fit(X_train_cv, y_train_cv,
                          batch_size=batch_size,
                          epochs=epochs,
                          verbose=verbose,
                          validation_data=(X_valid, y_valid),
                          callbacks=callbacks)
    return result

def data_augment_fit(model,callbacks,epochs = 100,batch_size = 32,verbose = 1):
    
    result = model.fit_generator(data_augmentation(X_train_cv,y_train_cv,batch_size = batch_size),
                          steps_per_epoch = len(X_train_cv)/batch_size,
                          epochs=epochs,
                          verbose=verbose,
                          validation_data=(X_valid, y_valid),
                          callbacks=callbacks)
    return result


In [ ]:
Cnnmodel = model_compile(cnnmodel(),lr = 0.0001,decay = 0, freezing_layers = None,weights_path = None)
callbacks = get_callbacks(filepath = 'model save/Model 7-2x filters Advanced CNN with DA and self TL/2018.1.2.hdf5', patience=100, save_best_only = False)
result = normal_fit(Cnnmodel,callbacks = callbacks,epochs = 100,batch_size=32,verbose=2)

In [ ]:
Cnnmodel = model_compile(cnnmodel(),lr = 0.0001,decay = 0, freezing_layers = 5,weights_path = 'model save/Model 7-2x filters Advanced CNN with DA and self TL/2018.1.2.hdf5')
callbacks = get_callbacks(filepath = 'model save/Model 7-2x filters Advanced CNN with DA and self TL/2018.1.2_stage2.hdf5', patience=30, save_best_only = True)
result = data_augment_fit(Cnnmodel,callbacks = callbacks,epochs = 100,batch_size=32,verbose=2)

In [ ]:
Cnnmodel.load_weights('model save/Model 7-2x filters Advanced CNN with DA and self TL/Model-7 cv1_stage1.hdf5')
Cnnmodel.evaluate(X_valid, y_valid)
#Cnnmodel.evaluate(X_train_cv, y_train_cv)

In [ ]:
def save_history(result,name = 'fit_history.csv',path = os.getcwd()):
    train_loss = result.history['loss']
    val_loss = result.history['val_loss']
    train_accuracy = result.history['acc']
    val_accuracy = result.history['val_acc']
    fit_history = pd.DataFrame({'train_loss':train_loss,'val_loss':val_loss,'train_accuracy':train_accuracy,'val_accuracy':val_accuracy})
    fit_history.to_csv(os.path.join(path,name))

In [ ]:
save_history(result_Cnn)

### Prediction

In [8]:
def Model7_CV(X_train,K=3):
    folds = list(StratifiedKFold(n_splits=K, shuffle=True,random_state = 3).split(X_train, target_train))
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    for j, (train_idx, valid_idx) in enumerate(folds):
        print('\n===================FOLD=',j+1)
        global X_train_cv
        X_train_cv = X_train[train_idx]
        global y_train_cv
        y_train_cv = target_train[train_idx]
        global X_valid
        X_valid = X_train[valid_idx]
        global y_valid
        y_valid= target_train[valid_idx]
        
        #Angle
#         X_angle_cv=X_angle[train_idx]
#         X_angle_hold=X_angle[test_idx]

        #define file path and get callbacks
        
        file_path_stage2 = 'model save/Model 8-Advanced CNN with DA(cv5)/Model-8 cv'+str(j+1)+'.hdf5'
        Cnnmodel = model_compile(cnnmodel(),lr = 0.0001,decay = 0, freezing_layers = None,weights_path = None)
        callbacks = get_callbacks(filepath = file_path_stage2, patience=30, save_best_only = True)
        result2 = data_augment_fit(Cnnmodel,callbacks = callbacks,epochs = 150,batch_size=32,verbose=2)
        print('Satge 2,done!')
#         callbacks = get_callbacks(filepath=file_path, patience=5)
#         gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
#         galaxyModel= getVggAngleModel()
#         galaxyModel.fit_generator(
#                 gen_flow,
#                 steps_per_epoch=24,
#                 epochs=100,
#                 shuffle=True,
#                 verbose=1,
#                 validation_data=([X_holdout,X_angle_hold], Y_holdout),
#                 callbacks=callbacks)

        #Getting the Best Model
        Cnnmodel.load_weights(filepath=file_path_stage2)
        #Getting Training Score
        score = Cnnmodel.evaluate(X_train_cv, y_train_cv)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = Cnnmodel.evaluate(X_valid, y_valid)
        print('Valid loss:', score[0])
        print('Valid accuracy:', score[1])

        #Getting validation Score.
        pred_valid=Cnnmodel.predict(X_valid)
        y_valid_pred_log[valid_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
#         temp_test=Cnnmodel.predict(X_test)
#         y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=Cnnmodel.predict(X_train)
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

#     y_valid_pred_log=y_valid_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_valid_pred_log

In [9]:
Model7_CV(X_train,K = 5)


===================FOLD= 1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 75, 75, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 71, 71, 128)       9728      
_________________________________________________________________
activation_1 (Activation)    (None, 71, 71, 128)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 35, 35, 128)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 35, 35, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 33, 33, 256)       295168    
_________________________________________________________________
activation_2 (Activation)    (None, 33, 33, 256)

Epoch 28/150
Epoch 00028: val_loss did not improve
 - 12s - loss: 0.3270 - acc: 0.8376 - val_loss: 0.3811 - val_acc: 0.7981
Epoch 29/150
Epoch 00029: val_loss improved from 0.33506 to 0.32480, saving model to model save/Model 8-Advanced CNN with DA(cv5)/Model-8 cv1.hdf5
 - 12s - loss: 0.3467 - acc: 0.8462 - val_loss: 0.3248 - val_acc: 0.8385
Epoch 30/150
Epoch 00030: val_loss did not improve
 - 12s - loss: 0.3258 - acc: 0.8498 - val_loss: 0.3616 - val_acc: 0.7981
Epoch 31/150
Epoch 00031: val_loss did not improve
 - 12s - loss: 0.3016 - acc: 0.8665 - val_loss: 0.3854 - val_acc: 0.8168
Epoch 32/150
Epoch 00032: val_loss did not improve
 - 12s - loss: 0.3219 - acc: 0.8516 - val_loss: 0.3438 - val_acc: 0.8230
Epoch 33/150
Epoch 00033: val_loss improved from 0.32480 to 0.29823, saving model to model save/Model 8-Advanced CNN with DA(cv5)/Model-8 cv1.hdf5
 - 12s - loss: 0.3209 - acc: 0.8513 - val_loss: 0.2982 - val_acc: 0.8571
Epoch 34/150
Epoch 00034: val_loss improved from 0.29823 to 0.29

Epoch 85/150
Epoch 00085: val_loss did not improve
 - 12s - loss: 0.2729 - acc: 0.8706 - val_loss: 0.3325 - val_acc: 0.8261
Epoch 86/150
Epoch 00086: val_loss did not improve
 - 12s - loss: 0.2348 - acc: 0.9024 - val_loss: 0.2898 - val_acc: 0.8602
Epoch 87/150
Epoch 00087: val_loss did not improve
 - 12s - loss: 0.2246 - acc: 0.8963 - val_loss: 0.2620 - val_acc: 0.8602
Epoch 88/150
Epoch 00088: val_loss did not improve
 - 12s - loss: 0.2598 - acc: 0.8813 - val_loss: 0.2777 - val_acc: 0.8447
Epoch 89/150
Epoch 00089: val_loss did not improve
 - 12s - loss: 0.2110 - acc: 0.9161 - val_loss: 0.2652 - val_acc: 0.8727
Epoch 90/150
Epoch 00090: val_loss did not improve
 - 12s - loss: 0.2091 - acc: 0.9092 - val_loss: 0.2477 - val_acc: 0.8944
Epoch 91/150
Epoch 00091: val_loss did not improve
 - 12s - loss: 0.1985 - acc: 0.9199 - val_loss: 0.2587 - val_acc: 0.8820
Epoch 92/150
Epoch 00092: val_loss did not improve
 - 12s - loss: 0.1921 - acc: 0.9260 - val_loss: 0.2697 - val_acc: 0.8665
Epoch 93

Epoch 6/150
Epoch 00006: val_loss improved from 0.54407 to 0.53886, saving model to model save/Model 8-Advanced CNN with DA(cv5)/Model-8 cv2.hdf5
 - 12s - loss: 0.5729 - acc: 0.6580 - val_loss: 0.5389 - val_acc: 0.7227
Epoch 7/150
Epoch 00007: val_loss improved from 0.53886 to 0.53091, saving model to model save/Model 8-Advanced CNN with DA(cv5)/Model-8 cv2.hdf5
 - 12s - loss: 0.5714 - acc: 0.6226 - val_loss: 0.5309 - val_acc: 0.7352
Epoch 8/150
Epoch 00008: val_loss improved from 0.53091 to 0.52050, saving model to model save/Model 8-Advanced CNN with DA(cv5)/Model-8 cv2.hdf5
 - 12s - loss: 0.5479 - acc: 0.6733 - val_loss: 0.5205 - val_acc: 0.7882
Epoch 9/150
Epoch 00009: val_loss improved from 0.52050 to 0.48095, saving model to model save/Model 8-Advanced CNN with DA(cv5)/Model-8 cv2.hdf5
 - 12s - loss: 0.5372 - acc: 0.6943 - val_loss: 0.4810 - val_acc: 0.7726
Epoch 10/150
Epoch 00010: val_loss improved from 0.48095 to 0.46453, saving model to model save/Model 8-Advanced CNN with DA

Epoch 57/150
Epoch 00057: val_loss did not improve
 - 12s - loss: 0.2482 - acc: 0.8807 - val_loss: 0.2384 - val_acc: 0.8941
Epoch 58/150
Epoch 00058: val_loss did not improve
 - 12s - loss: 0.2473 - acc: 0.8871 - val_loss: 0.2431 - val_acc: 0.8910
Epoch 59/150
Epoch 00059: val_loss did not improve
 - 12s - loss: 0.2250 - acc: 0.8963 - val_loss: 0.2116 - val_acc: 0.9034
Epoch 60/150
Epoch 00060: val_loss did not improve
 - 12s - loss: 0.2501 - acc: 0.8990 - val_loss: 0.3748 - val_acc: 0.8629
Epoch 61/150
Epoch 00061: val_loss did not improve
 - 12s - loss: 0.2603 - acc: 0.8826 - val_loss: 0.2057 - val_acc: 0.9252
Epoch 62/150
Epoch 00062: val_loss did not improve
 - 12s - loss: 0.2432 - acc: 0.8921 - val_loss: 0.2198 - val_acc: 0.9034
Epoch 63/150
Epoch 00063: val_loss did not improve
 - 12s - loss: 0.2269 - acc: 0.8937 - val_loss: 0.2518 - val_acc: 0.8629
Epoch 64/150
Epoch 00064: val_loss did not improve
 - 12s - loss: 0.2175 - acc: 0.8940 - val_loss: 0.2295 - val_acc: 0.9097
Epoch 65

...

Epoch 130/150
Epoch 00130: val_loss did not improve
 - 12s - loss: 0.1478 - acc: 0.9337 - val_loss: 0.2433 - val_acc: 0.9125
Epoch 131/150
Epoch 00131: val_loss did not improve
 - 12s - loss: 0.1601 - acc: 0.9314 - val_loss: 0.2198 - val_acc: 0.8969
Epoch 132/150
Epoch 00132: val_loss did not improve
 - 12s - loss: 0.1546 - acc: 0.9337 - val_loss: 0.2439 - val_acc: 0.9000
Epoch 133/150
Epoch 00133: val_loss did not improve
 - 12s - loss: 0.1597 - acc: 0.9298 - val_loss: 0.2368 - val_acc: 0.8844
Epoch 134/150
Epoch 00134: val_loss did not improve
 - 12s - loss: 0.1513 - acc: 0.9321 - val_loss: 0.2262 - val_acc: 0.9000
Epoch 135/150
Epoch 00135: val_loss did not improve
 - 12s - loss: 0.1358 - acc: 0.9436 - val_loss: 0.2208 - val_acc: 0.8969
Epoch 136/150
Epoch 00136: val_loss did not improve
 - 12s - loss: 0.1638 - acc: 0.9390 - val_loss: 0.2189 - val_acc: 0.8969
Epoch 137/150
Epoch 00137: val_loss did not improve
 - 12s - loss: 0.1459 - acc: 0.9451 - val_loss: 0.2258 - val_acc: 0.8969


Epoch 14/150
Epoch 00014: val_loss improved from 0.40012 to 0.39917, saving model to model save/Model 8-Advanced CNN with DA(cv5)/Model-8 cv5.hdf5
 - 12s - loss: 0.4095 - acc: 0.8078 - val_loss: 0.3992 - val_acc: 0.8156
Epoch 15/150
Epoch 00015: val_loss improved from 0.39917 to 0.36036, saving model to model save/Model 8-Advanced CNN with DA(cv5)/Model-8 cv5.hdf5
 - 12s - loss: 0.3884 - acc: 0.8240 - val_loss: 0.3604 - val_acc: 0.8313
Epoch 16/150
Epoch 00016: val_loss improved from 0.36036 to 0.35390, saving model to model save/Model 8-Advanced CNN with DA(cv5)/Model-8 cv5.hdf5
 - 12s - loss: 0.3835 - acc: 0.8330 - val_loss: 0.3539 - val_acc: 0.8438
Epoch 17/150
Epoch 00017: val_loss did not improve
 - 12s - loss: 0.3674 - acc: 0.8376 - val_loss: 0.4173 - val_acc: 0.7969
Epoch 18/150
Epoch 00018: val_loss did not improve
 - 12s - loss: 0.3722 - acc: 0.8246 - val_loss: 0.3738 - val_acc: 0.8250
Epoch 19/150
Epoch 00019: val_loss improved from 0.35390 to 0.34481, saving model to model s

Epoch 67/150
Epoch 00067: val_loss did not improve
 - 12s - loss: 0.2440 - acc: 0.8903 - val_loss: 0.2379 - val_acc: 0.9094
Epoch 68/150
Epoch 00068: val_loss did not improve
 - 12s - loss: 0.2168 - acc: 0.9062 - val_loss: 0.2384 - val_acc: 0.9125
Epoch 69/150
Epoch 00069: val_loss did not improve
 - 12s - loss: 0.2357 - acc: 0.8932 - val_loss: 0.2538 - val_acc: 0.8969
Epoch 70/150
Epoch 00070: val_loss improved from 0.22608 to 0.22396, saving model to model save/Model 8-Advanced CNN with DA(cv5)/Model-8 cv5.hdf5
 - 12s - loss: 0.2200 - acc: 0.9077 - val_loss: 0.2240 - val_acc: 0.9062
Epoch 71/150
Epoch 00071: val_loss did not improve
 - 12s - loss: 0.2218 - acc: 0.8949 - val_loss: 0.2995 - val_acc: 0.8750
Epoch 72/150
Epoch 00072: val_loss improved from 0.22396 to 0.21565, saving model to model save/Model 8-Advanced CNN with DA(cv5)/Model-8 cv5.hdf5
 - 12s - loss: 0.2153 - acc: 0.9039 - val_loss: 0.2156 - val_acc: 0.9219
Epoch 73/150
Epoch 00073: val_loss did not improve
 - 12s - loss

Epoch 128/150
Epoch 00128: val_loss improved from 0.20425 to 0.20352, saving model to model save/Model 8-Advanced CNN with DA(cv5)/Model-8 cv5.hdf5
 - 12s - loss: 0.1497 - acc: 0.9375 - val_loss: 0.2035 - val_acc: 0.9219
Epoch 129/150
Epoch 00129: val_loss did not improve
 - 12s - loss: 0.1665 - acc: 0.9260 - val_loss: 0.2157 - val_acc: 0.9125
Epoch 130/150
Epoch 00130: val_loss did not improve
 - 12s - loss: 0.1527 - acc: 0.9337 - val_loss: 0.2086 - val_acc: 0.9062
Epoch 131/150
Epoch 00131: val_loss did not improve
 - 12s - loss: 0.1464 - acc: 0.9405 - val_loss: 0.2139 - val_acc: 0.9125
Epoch 132/150
Epoch 00132: val_loss did not improve
 - 12s - loss: 0.1595 - acc: 0.9306 - val_loss: 0.2316 - val_acc: 0.9125
Epoch 133/150
Epoch 00133: val_loss did not improve
 - 12s - loss: 0.1753 - acc: 0.9154 - val_loss: 0.2339 - val_acc: 0.9125
Epoch 134/150
Epoch 00134: val_loss did not improve
 - 12s - loss: 0.1579 - acc: 0.9254 - val_loss: 0.2076 - val_acc: 0.9156
Epoch 135/150
Epoch 00135: va

0       2.634234e-05
1       7.572710e-12
2       8.207796e-01
3       1.029958e-04
4       1.034867e-02
5       1.000000e+00
6       9.873828e-01
7       8.719710e-01
8       1.457550e-11
9       1.286021e-01
10      3.451467e-01
11      2.154507e-04
12      9.107782e-01
13      8.817458e-01
14      4.322701e-03
15      5.364039e-02
16      5.240228e-01
17      6.339528e-03
18      4.478098e-01
19      5.596439e-01
20      2.318110e-02
21      3.361613e-01
22      4.370111e-08
23      9.999779e-01
24      7.067740e-09
25      9.999441e-01
26      9.404380e-02
27      1.573823e-03
28      5.041999e-01
29      7.440081e-06
            ...     
1574    2.094985e-01
1575    2.481993e-10
1576    1.271278e-07
1577    2.172237e-02
1578    6.413934e-02
1579    2.686754e-01
1580    5.664628e-02
1581    7.319692e-02
1582    5.183475e-01
1583    5.260556e-01
1584    1.192241e-04
1585    1.654384e-02
1586    5.180713e-01
1587    7.185864e-02
1588    5.936407e-01
1589    6.612022e-03
1590    1.654

### Submission

In [ ]:
def Evaluation_CV(X_train,K=3):
    folds = list(StratifiedKFold(n_splits=K, shuffle=True,random_state = 3).split(X_train, target_train))
    y_test_pred_log=0
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    for j, (train_idx, valid_idx) in enumerate(folds):
        print('\n===================FOLD=',j+1)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_valid = X_train[valid_idx]
        y_valid= target_train[valid_idx]
        
        #Angle
#         X_angle_cv=X_angle[train_idx]
#         X_angle_hold=X_angle[test_idx]

        #define file path and get callbacks
        file_path = 'model save/Model 6-Advanced CNN with DA and self TL(cv3)/Model-6 cv'+str(j+1)+'.hdf5'
        Cnnmodel=cnnmodel(input_shape = (75,75,3),lr = 0.0001)
        Cnnmodel.load_weights(file_path)


#         callbacks = get_callbacks(filepath=file_path, patience=5)
#         gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
#         galaxyModel= getVggAngleModel()
#         galaxyModel.fit_generator(
#                 gen_flow,
#                 steps_per_epoch=24,
#                 epochs=100,
#                 shuffle=True,
#                 verbose=1,
#                 validation_data=([X_holdout,X_angle_hold], Y_holdout),
#                 callbacks=callbacks)

        #Getting the Best Model
        #Getting Training Score
        print('cv'+str(j+1))
        score = Cnnmodel.evaluate(X_train_cv, y_train_cv)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = Cnnmodel.evaluate(X_valid, y_valid)
        print('Valid loss:', score[0])
        print('Valid accuracy:', score[1])
        print('\n')

        #Getting validation Score.
        pred_valid=Cnnmodel.predict(X_valid)
        y_valid_pred_log[valid_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
        temp_test=Cnnmodel.predict(X_test)
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=Cnnmodel.predict(X_train)
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

    y_valid_pred_log=y_valid_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_test_pred_log

In [ ]:
preds = Evaluation_CV(X_train,K=5)
sub = test[['id']]
sub['is_iceberg'] = preds
sub.to_csv('5cv.csv',index = False)

### Submission(min max median stacking)

In [10]:
def test_scores(K=5):
    y_test_pred_log=0
    test_df = test_data[['id']]
    for j in range(K):
        print('\n===================FOLD=',j+1)
        file_path = 'model save/Model 8-Advanced CNN with DA(cv5)/Model-8 cv'+str(j+1)+'.hdf5'
        Cnnmodel = model_compile(cnnmodel(),lr = 0.0001,decay = 0, freezing_layers = None,weights_path = file_path)

        #Getting Test Scores
        temp_test=Cnnmodel.predict(X_test)
        test_df['cv'+str(j+1)]=temp_test.reshape(temp_test.shape[0])
    return test_df

In [17]:
test_df = test_scores()


===================FOLD= 1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 75, 75, 3)         0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 71, 71, 128)       9728      
_________________________________________________________________
activation_21 (Activation)   (None, 71, 71, 128)       0         
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 35, 35, 128)       0         
_________________________________________________________________
dropout_31 (Dropout)         (None, 35, 35, 128)       0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 33, 33, 256)       295168    
_________________________________________________________________
activation_22 (Activation)   (None, 33, 33, 256)

c:\users\yltbe\anaconda3\envs\tensorflow_gpu\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()



===================FOLD= 2
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 75, 75, 3)         0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 71, 71, 128)       9728      
_________________________________________________________________
activation_25 (Activation)   (None, 71, 71, 128)       0         
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 35, 35, 128)       0         
_________________________________________________________________
dropout_37 (Dropout)         (None, 35, 35, 128)       0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 33, 33, 256)       295168    
_________________________________________________________________
activation_26 (Activation)   (None, 33, 33, 256)


===================FOLD= 5
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 75, 75, 3)         0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 71, 71, 128)       9728      
_________________________________________________________________
activation_37 (Activation)   (None, 71, 71, 128)       0         
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 35, 35, 128)       0         
_________________________________________________________________
dropout_55 (Dropout)         (None, 35, 35, 128)       0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 33, 33, 256)       295168    
_________________________________________________________________
activation_38 (Activation)   (None, 33, 33, 256)

In [18]:
test_df

,id,cv1,cv2,cv3,cv4,cv5
0,5941774d,5.428893e-02,6.673208e-02,7.094257e-02,3.609517e-03,4.734804e-02
1,4023181e,5.245490e-01,4.921556e-01,3.735768e-01,2.362068e-01,3.767341e-01
2,b20200e4,7.215895e-06,5.210542e-08,7.925581e-10,2.667083e-08,4.645425e-06
3,e7f018bb,9.979774e-01,9.999882e-01,9.976949e-01,9.999983e-01,9.999951e-01
4,4371c8c3,1.210454e-01,9.455159e-04,3.490464e-02,3.091512e-03,4.910483e-05
5,a8d9b1fd,8.201791e-02,1.096915e-01,1.754192e-02,6.308465e-02,6.289902e-02
6,29e7727e,2.341684e-01,1.582016e-01,4.415979e-02,3.304548e-01,1.907021e-01
7,92a51ffb,9.991114e-01,9.999897e-01,9.985918e-01,9.999380e-01,9.999481e-01
8,c769ac97,3.057454e-05,1.249167e-04,1.259599e-06,1.479310e-07,2.700044e-09
9,aee0547d,1.977355e-09,3.014829e-10,8.679121e-09,9.400190e-12,3.189674e-13


In [19]:
def get_sub_mmm(test_df):
    prediction = test_df
    prediction['is_iceberg'] = np.where(np.all(prediction[prediction.columns[1:]]>0.9,axis = 1),
                                    np.max(prediction[prediction.columns[1:]],axis = 1),
                                     np.where(np.all(prediction[prediction.columns[1:]]<0.1,axis = 1),np.min(prediction[prediction.columns[1:]],axis = 1),np.median(prediction[prediction.columns[1:]],axis = 1)))
    sub = prediction[['id','is_iceberg']]
    sub.to_csv('sub.csv',index = False)
    return sub

In [20]:
sub = get_sub_mmm(test_df)

c:\users\yltbe\anaconda3\envs\tensorflow_gpu\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [21]:
sub

,id,is_iceberg
0,5941774d,3.609517e-03
1,4023181e,3.767341e-01
2,b20200e4,7.925581e-10
3,e7f018bb,9.999983e-01
4,4371c8c3,3.091512e-03
5,a8d9b1fd,6.308465e-02
6,29e7727e,1.907021e-01
7,92a51ffb,9.999897e-01
8,c769ac97,2.700044e-09
9,aee0547d,3.189674e-13
